In [ ]:
#Machine learning code
#Written by Arun 
#Programming language: Python

import numpy as np
import scipy as sp
import math
import time
import itertools
import seaborn as sns
from numpy import linalg              
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split          
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.kernel_ridge import KernelRidge
from matplotlib import pyplot as plt
import sys, traceback
import warnings
import random
warnings.simplefilter("error")
np.seterr(divide='ignore', invalid='ignore')   #skips divide by zero , NaN errors
     
start = time.time()			  #timer
file1 = open("data-X.dat",      "r")         
gwfile= open("data-gw.dat",   "r")        

#reading data file
data   = []
for line in file1:
	data.append( [float(n) for n in line.split()])

#reading GW file
egap_gw= []
for line2 in gwfile:
	egap_gw.append(line2.split())

#convert to numpy arrays
data    = np.asarray(data,    dtype=float) 
egap_gw = np.asarray(egap_gw, dtype=float)

print("")
print("="*80)
print("                               MACHINE LEARNING                             ")
print("="*80)

 
#---------------Normalize the given data------------------
print("Data...")
data_nrow, data_ncol   = data.shape
#print(data.shape)

# print("Normalization (Centering and Scaling). . .")
# #centering
# data = StandardScaler().fit_transform(data)

# #scale 0-1
# data = MinMaxScaler().fit_transform(data)                       
# print("")

print("="*80)

#-------------------- primary features--------------------
prim_data  = np.copy(data)
print("Primary features           :", prim_data.shape)
print("")

#------------------- #We dont use feature selection here!
prim_nrow, prim_ncol = prim_data.shape

# #---------------- Correlation co-efficient ------- 
# print("Highly correlated primary features.......")
# corrmax = 0.99
# deletep = []
# for _prim_ncol in range(prim_ncol):
# 	column1 = (prim_data[:,[_prim_ncol]]).T
# 	for _prim_ncol_new in range(prim_ncol):
# 		column2 = (prim_data[:,[_prim_ncol_new]]).T
# 		if _prim_ncol < _prim_ncol_new:
# 			corr    = np.corrcoef(column1, column2)[0, 1]
# 			if corr > corrmax:
# 				print(_prim_ncol, " & ", _prim_ncol_new, ": corr=", corr)
# 				deletep.append(_prim_ncol_new)

# deletep = np.unique(deletep)
# if len(deletep) > 0:
# 	print(len(deletep)," indices to be deleted:", deletep)
# 	prim_data = np.delete(prim_data, deletep, 1)
# 	print("")
# 	print("Primary features (modified):", prim_data.shape)
# 	prim_nrow, prim_ncol = prim_data.shape
# else:
# 	print("No feature to be deleted.")
# print("")

X = np.copy(prim_data)
y = np.copy(egap_gw)
print(" ")

print("="*80)
print("Boxplot for GW data:")
plt.figure(0)
bplot    = plt.boxplot(y)
print("Medians       :", [item.get_ydata()[0] for item in bplot['medians']])
#fliers  = [item.get_ydata() for item in bplot['fliers']]
fliers   = []
for _fliers in bplot['fliers']:
	_fliers  = _fliers.get_ydata()
	fliers.append(_fliers)

fliers  = np.asarray(fliers)
print("fliers        :", fliers.ravel())
print("")
print("Figure: boxplot_GW_egap.png")
plt.savefig("boxplot_GW_egap.png")
print("")

# print("Removing outliers from GW data:")
# outlier_index = []
# _ycounter     = 0
# for _y in y.ravel():
# 	for _fliers in fliers.ravel():
# 		if float(_y) == float(_fliers):	     #outlier listed
# 			outlier_index.append(_ycounter)
# 	_ycounter += 1

# outlier_index = np.unique(outlier_index)
# if len(outlier_index) > 0:
# 	for _outlier_index in outlier_index.ravel():
# 		print(_outlier_index, "    :   ", y[_outlier_index])

# 	X 	  = np.delete(X, outlier_index, 0)		#deleting outliers from y (and hence from X)
# 	y 	  = np.copy(np.delete(y, outlier_index, 0))
# 	prim_data = np.delete(prim_data, outlier_index, 0)      #remove corresponding samples from prim_data
# #	comp_data = np.delete(comp_data, outlier_index, 0)

# 	print("")
# 	print("X                 :", X.shape)
# 	print("y                 :", y.shape)

# #	comp_nrow, comp_ncol 	= comp_data.shape
# 	prim_nrow, prim_ncol 	= prim_data.shape
# 	print("Updated X, y & prim_data!")
	
# else:
# 	print("Outliers not found!")
# print("Done")
# print("")
# X                       = np.copy(X)
X                       = np.copy(prim_data)
prim_nrow, prim_ncol    = prim_data.shape
print("Best primary data: best-prim.dat")
np.savetxt('best-prim.dat', prim_data, fmt='%15.8f', 
    delimiter=' ', newline='\n', header='', footer='', comments='# ')


print("="*80)
print("PRIMARY feature combinations . . . !")
test_size                 = 0.10
_n_estimators             = 100
_ntrials                  = 50
nplots                    = 2            #2 plots/feature combination/algorithm
comb_data_set  	          = ['prim_data']


for _comb_data_set in range(len(comb_data_set)):
    print(comb_data_set[_comb_data_set],":")
    if _comb_data_set == 0:
        comb_data = np.copy(prim_data)
        print("PRIMARY")
    else:
        print("Not PRIMARY . . . stopping!")
        sys.exit()
    
    comb_nrow, comb_ncol = comb_data.shape
    ctr         = 0
    nfeature    = np.arange(comb_ncol)    
    
    regression = ["KRR"]
    #regression = ["SVR"]
    nctr       = 0
    for _nfeature in nfeature: # 1 to n
        print("="*80)
        nctr += 1
        for tag in regression:
            nfeature_list = list(itertools.combinations(nfeature, _nfeature+1))    #nCr combinations (ex: [1,2], [.,.], etc for 2 feature combinations)
            print("#features (r):", nctr, "#nCr :", len(nfeature_list))
            r2_comb      = []                                                              
            rmse_comb    = []                                                               
            mae_comb     = []                                                               
            feature_comb = []  
            r2_train_comb      = []                                                              
            rmse_train_comb    = []                                                               
            mae_train_comb     = []                                                               
            feature_train_comb = []             
            print("")
            print(tag, ":")
            
            if len(nfeature_list[0]) != 8:
                print("len(nfeature_list[0])", len(nfeature_list[0]))
                continue
            
            for _nfeature_list in nfeature_list:                                            #each combination
                    X        = np.copy(comb_data[:, _nfeature_list])         
                    comb_nrow, comb_ncol = X.shape      
                  
                    if tag ==  "LR":
                        estm            = LinearRegression()
                    elif tag == "Lasso":
                        estm            = LassoCV(tol=0.001,max_iter=10000000)
                    elif tag == "KRR":
                        estm            = GridSearchCV(KernelRidge(kernel='rbf'), cv=5,
                        param_grid={"alpha": [1e0, 0.5, 0.1, 0.05, 1e-2, 0.005, 1e-3], "gamma": np.logspace(-4, 4, 20)})
                    elif tag == "SVR":
                        estm            = GridSearchCV(SVR(kernel='rbf', gamma=0.1), cv=5,
                            param_grid={"C": [1e0, 1e1, 1e2, 1e3],
                            "gamma": np.logspace(-2, 2, 5)})
                    elif tag == "AdaBoost":
                        estm            = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4), 
                            n_estimators=100, random_state=int(np.random.rand()*100.0)) 
                    elif tag == "Bagging":
#                         estm            = BaggingRegressor(DecisionTreeRegressor())
                        estm            = BaggingRegressor(DecisionTreeRegressor(max_depth=4), 
                            n_estimators=_n_estimators, random_state=int(np.random.rand()*100.0))                         
                    else:
                        print("Unknown algorithm")
                        sys.exit()
                
                    ntrials  = _ntrials
                    r2_array = []
                    mse_array= []
                    mae_array= []
                    r2_train_array = []
                    mse_train_array= []
                    mae_train_array= []                    
                    #print("Averaging over", ntrials, "trials")
                    for _rs in range(ntrials):
                        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, 
                            random_state=int(np.random.rand()*100.0))             #train & test. 
#                         if tag == "Lasso" or tag == "SVR" or tag == "AdaBoost" or tag == "Bagging":
#                             y_pred          = estm.fit(X_train, y_train.ravel()).predict(X_test)
#                         else:
                        y_pred              = estm.fit(X_train, y_train.ravel()).predict(X_test)  
                        y_pred_train        = estm.fit(X_train, y_train.ravel()).predict(X_train)      
                        mse                 = mean_squared_error(y_test, y_pred)
                        r2                  = r2_score(y_test, y_pred)
                        mae                 = mean_absolute_error(y_test, y_pred)
                        
                        mse_train           = mean_squared_error(y_train, y_pred_train)
                        r2_train            = r2_score(y_train, y_pred_train)
                        mae_train           = mean_absolute_error(y_train, y_pred_train)

                        if r2 > 0:
                            r2_array.append(r2)            
                            mse_array.append(mse)          
                            mae_array.append(mae)   

                            r2_train_array.append(r2_train)            
                            mse_train_array.append(mse_train)          
                            mae_train_array.append(mae_train) 
                            
                            avg_rmse = np.mean(np.sqrt(mse_array))
                            avg_r2   = np.mean(r2_array)     
                            avg_mae  = np.mean(mae_array)   
                            
                            avg_rmse_train = np.mean(np.sqrt(mse_train_array))
                            avg_r2_train   = np.mean(r2_train_array)     
                            avg_mae_train  = np.mean(mae_train_array)                            

                    rmse_comb.append(avg_rmse)                     #metrics for given r.
                    r2_comb.append(avg_r2)
                    mae_comb.append(avg_mae)
                    feature_comb.append(_nfeature_list) 
                    
                    rmse_train_comb.append(avg_rmse_train)                     #metrics for given r.
                    r2_train_comb.append(avg_r2_train)
                    mae_train_comb.append(avg_mae_train)
                                     
                    if len(r2_comb) == 0:
                        print("Negative R2")
#                     else:
#                         print("feature_comb:", feature_comb)
                                                
            #based on max. R2 (RMSE min. R2 max and MAE min are always found different)
            index2       = list(r2_comb).index(max(r2_comb))
            best_rmse    = rmse_comb[index2]
            best_mae     = mae_comb[index2]
            best_r2      = r2_comb[index2]
            
            print("Averaged values over", ntrials, " cycles:")
            print("Best features:", feature_comb[index2])
            print("Best R2   test/train   :", round(best_r2,2),   "/", round(r2_train_comb[index2],2))
            print("Best rmse test/train   :", round(best_rmse,2), "/", round(rmse_train_comb[index2],2))
            print("Best mae  test/train   :", round(best_mae,2),  "/", round(mae_train_comb[index2],2))

#if required, the following canbe averaged.
#             array_test  = np.array([y_test.ravel(), y_pred.ravel()])
#             array_train = np.array([y_train.ravel(), y_pred_train.ravel()])
#             print("Test  St.err(eV):", np.std(array_test)/np.sqrt(np.shape(X_test)[0]))
#             print("Train St.err(eV):", np.std(array_train)/np.sqrt(np.shape(X_train)[0]))
            best_features   = feature_comb[index2]
            
            
#             best_rmse_nCr.append(rmse_array[index2])       #these can be fixed at the end.
#             best_mae_nCr.append(mae_array[index2])
#             best_r2_nCr.append(r2_array[index2])
#             best_feature_nCr.append(feature_comb[index2])             
                               
            print("best_features :", best_features)
            X_model         = np.copy(comb_data[:, best_features])
            #X               = np.copy(comb_data[:, _nfeature_list])
            
            print("X_model", X_model.shape)
            print("y",       y.shape)
              
#             print("#nfeature:", ctr+1, "alg:", tag, ". . . scatter plots (max. 2) . . .")
#             ctr          = 0
#             plot_counter = 0
#             for _rs in range(5 * ntrials):
#                         X_train, X_test, y_train, y_test = train_test_split(X_model, y, test_size=test_size, 
#                             random_state=int(np.random.rand() * 100.0))    #train & test. 
#                         if tag == "Lasso" or tag == "SVR" or tag == "AdaBoost" or tag == "Bagging":
#                             y_pred     = estm.fit(X_train, y_train.ravel()).predict(X_test)
#                             #y_pred     = estm.fit(X_train, y_train.ravel()).predict(X_test)
#                         else:
#                             y_pred     = estm.fit(X_train, y_train).predict(X_test)    
#                         rmse           = np.sqrt(mean_squared_error(y_test, y_pred))
#                         r2             = r2_score(y_test, y_pred)
#                         mae            = mean_absolute_error(y_test, y_pred)
                        
#                         lower_rmse     = best_rmse  - 0.1   
#                         upper_rmse     = best_rmse  + 0.1    
                        
#                         lower_mae      = best_mae   - 0.1    
#                         upper_mae      = best_mae   + 0.1  
                        
#                         lower_r2       = best_r2    - 0.1    
#                         upper_r2       = best_r2    + 0.1                                                                    
#                         rmse           = np.sqrt(mse)  
                        
#                         if ((rmse >= lower_rmse) and (rmse <= upper_rmse)) and ((mae >= lower_mae) and (mae <= upper_mae)):
#                             if (r2 >= lower_r2) and (r2 <= upper_r2):
#                                 plot_counter += 1
#                                 #print(tag, " (limit): rmse, r2, mae", np.sqrt(mse), r2, mae)
#                                 array_test  = np.array([y_test.ravel(), y_pred.ravel()])
#                                 array_train = np.array([y_train.ravel(), y_pred_train.ravel()])

#                                 if estm == "Lasso" or estm == "SVR" or estm == "AdaBoost":
#                                     y_pred_train    = estm.fit(X_train, y_train.ravel()).predict(X_train)
#                                 else:
#                                     y_pred_train    = estm.fit(X_train, y_train.ravel()).predict(X_train)  

#                                 rmse_train      = np.sqrt(mean_squared_error(y_train, y_pred_train))
#                                 r2_train        = r2_score(y_train, y_pred_train)
#                                 mae_train       = mean_absolute_error(y_train, y_pred_train)
#                                 print("Test  RMSE (eV) :", rmse,       " R2 :", r2,       " MAE (eV):", mae)
#                                 print("Train RMSE (eV) :", rmse_train, " R2 :", r2_train, " MAE (eV):", mae_train)
#                                 print("Test  St.err(eV):", np.std(array_test)/np.sqrt(np.shape(X_test)[0]))
#                                 print("Train St.err(eV):", np.std(array_train)/np.sqrt(np.shape(X_train)[0]))
                                  
#                                 plt.figure(int(np.random.rand())*1000)
#                                 plt.scatter(y_train, y_pred_train, s=70.0, c="red",  alpha=0.5)
#                                 plt.scatter(y_test, y_pred,        s=70.0, c="blue", alpha=0.5)
#                                 plt.plot([0.0, 6.0], [0.0, 6.0], color='white', linestyle='--', linewidth=4)
#                                 axes = plt.gca()     #the best way to set lim & labels
#                                 axes.set_xlim([1.5,5])
#                                 axes.set_ylim([1.5,5])
#                                 axes.set_ylabel("Predicted gap (eV)")
#                                 axes.set_xlabel("True gap (eV)")
#                                 axes.set_title("{}_{}_{}_{}_{}_{}".format(nctr,tag,
#                                     round(rmse_train,2), round(np.sqrt(mse),2), round(r2_train,2), round(r2,2)))
#                                 plt.show()
                                                            
#                                 test = np.concatenate((y_test.reshape(len(y_test),1), 
#                                         y_pred.reshape(len(y_test),1)), axis=1)
#                                 train = np.concatenate((y_train.reshape(len(y_train),1), 
#                                         y_pred_train.reshape(len(y_pred_train),1)), axis=1)
#                                 np.savetxt("case1-{}_{}_train_{}_{}_{}_r{}.{}".format(nctr,tag,_n_estimators,ntrials,test_size,ctr+1,plot_counter), train)
#                                 np.savetxt("case1-{}_{}_test_{}_{}_{}_r{}.{}".format(nctr,tag,_n_estimators,ntrials,test_size,ctr+1,plot_counter), test)
#                                 print("Saved files : case1-{}_{}_train_{}_{}_{}_r{}.{} & case1-{}_{}_test_{}_{}_{}_r{}.{}".
#                                       format(nctr,tag,_n_estimators,ntrials,test_size,ctr+1,plot_counter,nctr, tag,_n_estimators,ntrials,test_size,ctr+1,plot_counter))
                                             
#                                 if plot_counter == (nplots + 1):
#                                     print("Already plotted", plot_counter, "plots. Quitting")
#                                     break

        ctr         += 1
#close files.
file1.close()  
gwfile.close() 

#run time
end  = time.time()
m, s = divmod(end - start, 60)
h, m = divmod(m, 60)
print("Run time: %d:%02d:%02d" % (h, m, s)) 



                               MACHINE LEARNING                             
Data...
Primary features           : (70, 15)

 
Boxplot for GW data:
Medians       : [2.4012500000000001]
fliers        : [ 1.8312  1.7317  1.6075  1.5867  3.3956  3.3659  3.3072  3.2661  3.2553
  3.2203]

Figure: boxplot_GW_egap.png

Best primary data: best-prim.dat
PRIMARY feature combinations . . . !
prim_data :
PRIMARY
#features (r): 1 #nCr : 15

KRR :
len(nfeature_list[0]) 1
#features (r): 2 #nCr : 105

KRR :
len(nfeature_list[0]) 2
#features (r): 3 #nCr : 455

KRR :
len(nfeature_list[0]) 3
#features (r): 4 #nCr : 1365

KRR :
len(nfeature_list[0]) 4
#features (r): 5 #nCr : 3003

KRR :
len(nfeature_list[0]) 5
#features (r): 6 #nCr : 5005

KRR :
len(nfeature_list[0]) 6
#features (r): 7 #nCr : 6435

KRR :
len(nfeature_list[0]) 7
#features (r): 8 #nCr : 6435

KRR :
